# GigaScience 크롤러

- 2012 ~ 2019년 (vol.1 ~ vol.27) 게재 논문 저자와 이메일을 크롤링하는 크롤러입니다.
- 크롤링 결과는 'Platform', 'Year', 'Volume', 'Author', 'Email', 'URL'의 6개 칼럼으로 구성된 데이터프레임으로 정리하였습니다.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [73]:
df = pd.DataFrame(columns = ['Platform', 'Year', 'Volume', 'Author', 'Email', 'URL'])

driver = webdriver.Chrome('../chromedriver')

root = 'https://academic.oup.com/gigascience/search-results?sort=Date+%e2%80%93+Newest+First&f_ContentType=Journal+Article&fl_SiteID=5589&rg_ArticleDate=01%2f01%2f1666+TO+12%2f31%2f2019&page='

for i in range(1, 35):   #34페이지까지 있음
    driver.get(root + str(i))
    articles = driver.find_elements_by_class_name('article-link')
    article_urls = [article.get_attribute('href') for article in articles]
    for url in article_urls:
        driver.get(url)
        authors = driver.find_elements_by_class_name('al-author-name-more')
        show_more = driver.find_elements_by_class_name('meta-authors--etal.is-active')
        if len(show_more) != 0:
            show_more[0].click()
        for name in authors:
            email_button = name.find_elements_by_class_name('icon-general-mail')
            if len(email_button) != 0:
                name.click()
                time.sleep(1)
                text = name.text
                author = text.split('Correspondence address.')[0].split('\n')[0]
                email = text.split('Correspondence address.')[-1].split('mail: ')[-1].split('\n')[0]
                citation = driver.find_element_by_class_name('ww-citation-primary').text
                year = citation.split(', ')[3].split()[-1]
                volume = citation.split(', ')[1].split()[-1]
                df = df.append({'Platform': 'GigaScience', 'Year': year, 'Volume': volume, 'Author': author, 'Email': email, 'URL': url}, ignore_index = True)
                elem = driver.find_element_by_class_name('center-inner-row')   #저자 상세보기 팝업 제거
                elem.click()
print('finished')

finished


In [74]:
df

,Platform,Year,Volume,Author,Email,URL
0,GigaScience,2019,8,Yuanfang Guan,gyuanfan@umich.edu,https://academic.oup.com/gigascience/article/8...
1,GigaScience,2019,8,Han Ming Gan,han.gan@deakin.edu.au,https://academic.oup.com/gigascience/article/8...
2,GigaScience,2019,8,Alexandra Pavlova,alexandra.pavlova@monash.edu,https://academic.oup.com/gigascience/article/8...
3,GigaScience,2019,8,Namshin Kim,deepreds@kribb.re.kr,https://academic.oup.com/gigascience/article/8...
4,GigaScience,2019,8,Johan Dahlberg,johan.dahlberg@medsci.uu.se,https://academic.oup.com/gigascience/article/8...
...,...,...,...,...,...,...
869,GigaScience,2012,1,Dawn Field,Dawn Field,https://academic.oup.com/gigascience/article/1...
870,GigaScience,2012,1,J Gregory Caporaso,J Gregory Caporaso,https://academic.oup.com/gigascience/article/1...
871,GigaScience,2012,1,Jonathan A Eisen,Jonathan A Eisen,https://academic.oup.com/gigascience/article/1...
872,GigaScience,2012,1,Laurie Goodman,Laurie Goodman,https://academic.oup.com/gigascience/article/1...


In [41]:
df.to_csv('GigaScience.csv', encoding = 'utf-8')